Multi-Target DMRG ITensor Version: Open U(1) symmetry.

In [1]:
using ITensors
using KrylovKit
using Einsum
using TensorOperations
const nexcit = 3
const maxdim = 40
const cutoff = 0
const virind = Index(QN()=>nexcit;tags="v")
function get_phi0(psi,b)
    T = psi[b]*psi[b+1]
    s1,s2 =  findinds(T;tags="Site")
    l = findinds(T;tags="Link")
    if length(l) == 2
        A = array(T,s1,s2,l[1],l[2],virind)
        phi0 = copy(A[:,:,:,:,1])
        return ITensor(phi0,s1,s2,l[1],l[2])
    else
        A = array(T,s1,s2,l[1],virind)
        phi0 = copy(A[:,:,:,1])
        return ITensor(phi0,s1,s2,l[1])
    end
end
function get_phi_tensor(phi_list,psi,b)
    T = psi[b]*psi[b+1]
    s1,s2 =  findinds(T;tags="Site")
    l = findinds(T;tags="Link")
    if length(l) == 2
        AT = array(T,s1,s2,l[1],l[2],virind)
        for n = 1:nexcit
            AT[:,:,:,:,n] = array(phi_list[n],s1,s2,l[1],l[2])
        end
        return ITensor(AT,s1,s2,l[1],l[2],virind)
    else
        AT = array(T,s1,s2,l[1],virind)
        for n = 1:nexcit
            AT[:,:,:,n] = array(phi_list[n],s1,s2,l[1])
        end
        return ITensor(AT,s1,s2,l[1],virind)
    end
end
function Tensor_inner(T1,T2)
    if length(size(T1)) == 2
        @einsum T[m,n] := T1[i,m]*T2[i,n]
        return tr(T)
    else
        @einsum T[m,n] := T1[i,j,m]*T2[i,j,n]
        return tr(T)
    end
end
function eigenvector_normalize(T)
    s = findinds(T;tags="Site")
    l = findinds(T;tags="Link")
    if length(l) == 2
        A = array(T,s[1],l[1],l[2],virind)
        for n = 1:nexcit
            for m = 1:n-1
                A[:,:,:,n] = A[:,:,:,n]-Tensor_inner(A[:,:,:,n],A[:,:,:,m])*A[:,:,:,m]
            end
            A[:,:,:,n] = A[:,:,:,n]/sqrt(Tensor_inner(A[:,:,:,n],A[:,:,:,n]))
        end
        return ITensor(A/sqrt(nexcit),s[1],l[1],l[2],virind)
    else
        A = array(T,s[1],l[1],virind)
        for n = 1:nexcit
            for m = 1:n-1
                A[:,:,n] = A[:,:,n]-Tensor_inner(A[:,:,n],A[:,:,m])*A[:,:,m]
            end
            A[:,:,n] = A[:,:,n]/sqrt(Tensor_inner(A[:,:,n],A[:,:,n]))
        end
        return ITensor(A/sqrt(nexcit),s[1],l[1],virind)
    end
end
function multi_replacebond!(psi,b,phi,ha)
    if ha == 1
        s =  findinds(psi[b];tags="Site")
        if b == 1
            indsb = s
        else
            indsb = (s,linkind(psi,b-1))
        end
        L,R,spec = ITensors.factorize(phi,indsb;which_decomp="svd",maxdim,cutoff,ortho="left",tags=tags(linkind(psi,b)))
        psi[b] = L
        R = eigenvector_normalize(R)
        psi[b+1] = R
    elseif ha == 2
        s =  findinds(psi[b];tags="Site")
        if b == 1
            indsb = (s,virind)
        else
            indsb = (s,linkind(psi,b-1),virind)
        end
        L,R,spec = ITensors.factorize(phi,indsb;which_decomp="svd",maxdim,cutoff,ortho="right",tags=tags(linkind(psi,b)))
        L = eigenvector_normalize(L)
        psi[b] = L
        psi[b+1] = R
    end
    if ha == 1
        ITensors.leftlim(psi) == b - 1 && ITensors.setleftlim!(psi, ITensors.leftlim(psi) + 1)
        ITensors.rightlim(psi) == b + 1 && ITensors.setrightlim!(M, ITensors.rightlim(psi) + 1)
        psi[b + 1] ./= norm(psi[b + 1])
    elseif ha == 2
        ITensors.leftlim(psi) == b && ITensors.setleftlim!(psi, ITensors.leftlim(psi) - 1)
        ITensors.rightlim(psi) == b + 2 && ITensors.setrightlim!(psi, ITensors.rightlim(psi) - 1)
        psi[b] ./= norm(psi[b])
    end
    return spec
end
function HamiltonianMultiply_matrix(P,phi0)
    itensor_map = Union{ITensor,ITensors.OneITensor}[lproj(P)]
    append!(itensor_map, P.H[ITensors.site_range(P)])
    Hv = rproj(P)
    for it in itensor_map
        Hv *= it
    end
    eigenvalue,eigenvector = eigen(Hv)
    eigenvalue = diag(eigenvalue)
    ind_phi = inds(phi0)
    if length(ind_phi) == 3
        e = []
        vec = []
        A = Array(eigenvector,(ind_phi[1],ind_phi[2],ind_phi[3],findinds(eigenvector;tags="eigen")))
        for i = 1:size(eigenvalue,1)
            phi = ITensor(A[:,:,:,i],ind_phi)
            if flux(phi)==flux(phi0)
                push!(e,real(eigenvalue[i]))
                push!(vec,phi)
            end
        end
    else
        e = []
        vec = []
        A = Array(eigenvector,(ind_phi[1],ind_phi[2],ind_phi[3],ind_phi[4],findinds(eigenvector;tags="eigen")))
        for i = 1:size(eigenvalue,1)
            phi = ITensor(A[:,:,:,:,i],ind_phi)
            if flux(phi)==flux(phi0)
                push!(e,real(eigenvalue[i]))
                push!(vec,phi)
            end
        end
    end
    order_e = sortperm(real(e))
    e = e[order_e]
    vec = vec[order_e]
    return e,vec
end
let
    #define MPO and initial MPS state
    N = 100
    sites = siteinds("S=1/2",N;conserve_qns=true)
    os = OpSum()
    for j in 1:(N-1)
        os .+= 0.5,"S+",j,"S-",j+1
        os .+= 0.5,"S-",j,"S+",j+1
        os .+= "Sz",j,"Sz",j+1
    end
    H = MPO(os,sites)
    state = [isodd(n) ? "Up" : "Dn" for n in 1:N]
    # psi0 = MPS(sites,state)
    psi0 = randomMPS(sites,state;linkdims=20)
    #set DMRG parameters
    nsweep = 4
    #do DMRG calculation
    PH = ProjMPO(H) #ProjMPO is a type of ITensor
    psi = copy(psi0)
    N = length(psi)
    if !isortho(psi) || orthoCenter(psi)!=1
        psi=ITensors.orthogonalize!(PH,psi,1) # set the orthogonal center at the first site
    end
    slinds = inds(psi[1])
    a1 = array(psi[1],slinds[1],slinds[2])
    a2 = zeros(ComplexF64,(dim(slinds[1]),dim(slinds[2]),nexcit))
    for n = 1:nexcit
        a2[:,:,n] = a1[:,:]
    end
    # psi[1] = ITensor(a2,slinds[1],slinds[2],virind)
    psi[1] = randomITensor(QN("Sz",0),slinds[1],slinds[2],virind)
    PH = position!(PH,psi,1) #Effective Hamiltonian for the first-two sites
    energy = 0.0
    for sw in 1:nsweep
        for (b,ha) in sweepnext(N)
            PH = position!(PH,psi,b) #Effective Hamiltonian for the two sites connected by the bond we want to optimaize
            phi0 = get_phi0(psi,b)
            vals,vecs = eigsolve(PH,phi0,nexcit,:SR;tol=1e-12,ishermitian=true,eager=true) #get the ground eigenvalue and the ground eigenstate of the coarsed site
            if length(vals) < nexcit
                vals,vecs = HamiltonianMultiply_matrix(PH,phi0)
            end
            energy = copy(vals[1:nexcit])
            phi_list = copy(vecs[1:nexcit])
            phi = get_phi_tensor(phi_list,psi,b)
            spec = multi_replacebond!(psi,b,phi,ha)
            println("site $b : energy = $energy linkdim = $(linkdim(psi,b)) trunc = $(spec.truncerr)")
        end
        println("After sweep $sw energy=$energy")
    end
    psi_list = Vector{Any}(undef, nexcit)
    for n = 1:nexcit
        psin = copy(psi)
        s = findinds(psi[1];tags="Site")
        l = findinds(psi[1];tags="Link")
        A = array(psi[1],s[1],l[1],virind)
        an = copy(A[:,:,n])
        Tn = ITensor(an,s[1],l[1])
        psin[1] = Tn
        psi_list[n] = psin
        println("The $(n-1)th excited state energy is: $(inner(psi_list[n]',H,psi_list[n])/inner(psi_list[n]',psi_list[n]))")
    end
end     
            

site 1 : energy = [-3.371873358365453, -2.8576743627622934, -2.4045577394996944] linkdim = 2 trunc = 0.0
site 2 : energy = [-3.7410958805046146, -3.335124078405168, -3.1404097378851485] linkdim = 4 trunc = 0.0
site 3 : energy = [-4.682050056834507, -4.171303122097819, -3.9783898589608526] linkdim = 8 trunc = 0.0
site 4 : energy = [-5.115963354984327, -4.774567658997316, -4.551846305313352] linkdim = 16 trunc = 0.0
site 5 : energy = [-5.717010945129126, -5.241862102402254, -5.124039462954876] linkdim = 32 trunc = 0.0
site 6 : energy = [-6.445733256317224, -6.048470826201952, -5.736773892721002] linkdim = 40 trunc = 1.0674505994142772e-11
site 7 : energy = [-6.518529538581997, -6.161841352059313, -5.9205578794418825] linkdim = 40 trunc = 7.768198323533465e-10
site 8 : energy = [-6.67627351894333, -6.331374016445386, -6.289919870189255] linkdim = 40 trunc = 1.2386849467783936e-9
site 9 : energy = [-7.043870936938269, -6.7559323271028076, -6.426752728528133] linkdim = 40 trunc = 6.77068703

site 67 : energy = [-32.18720757265802, -32.05159207844491, -31.975958655709807] linkdim = 40 trunc = 5.1177865126715265e-9
site 68 : energy = [-32.44091757785461, -32.31779233254546, -32.24673406528139] linkdim = 40 trunc = 2.4334994236440132e-8
site 69 : energy = [-33.0546404870893, -32.94815080785091, -32.873207350533185] linkdim = 40 trunc = 1.6859438867612797e-8
site 70 : energy = [-33.25796133356873, -33.15434335017742, -33.07872606261011] linkdim = 40 trunc = 3.809320398269846e-8
site 71 : energy = [-33.44054071779378, -33.34196440401314, -33.27270791694236] linkdim = 40 trunc = 6.472772124437541e-8
site 72 : energy = [-33.730098825591504, -33.635063440813546, -33.56590935120595] linkdim = 40 trunc = 9.010530650969347e-8
site 73 : energy = [-34.04011124033988, -33.941534691395944, -33.87998686723456] linkdim = 40 trunc = 3.5554530490006615e-7
site 74 : energy = [-34.625410959710415, -34.49099503498313, -34.45163471898136] linkdim = 40 trunc = 7.322001164536839e-7
site 75 : energ

┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31


site 98 : energy = Any[-44.09490829550406, -43.984127455403424, -43.92186316258339] linkdim = 12 trunc = 0.0


┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31


...

┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31


site 97 : energy = Any[-44.1269162588602, -44.086144679339824, -44.038486332145496] linkdim = 8 trunc = 0.0
site 96 : energy = [-44.12691808116408, -44.08614668780535, -44.03848689889348] linkdim = 16 trunc = 0.0
site 95 : energy = [-44.12691889285017, -44.08614723112929, -44.038487513341686] linkdim = 32 trunc = 0.0
site 94 : energy = [-44.12692521733428, -44.08615089152642, -44.03849118688097] linkdim = 40 trunc = 1.4028126815060002e-12
site 93 : energy = [-44.12692772940791, -44.08615323026799, -44.03849380896866] linkdim = 40 trunc = 6.717243620073865e-11
site 92 : energy = [-44.12693910584575, -44.086159915702, -44.038501366984875] linkdim = 40 trunc = 1.541829804423732e-10
site 91 : energy = [-44.12694668685905, -44.08616367201254, -44.03850512087594] linkdim = 40 trunc = 7.709894363088364e-10
site 90 : energy = [-44.12696274070281, -44.08617443195469, -44.03851698500238] linkdim = 40 trunc = 1.0458533879603265e-9
site 89 : energy = [-44.12697324829356, -44.086180479413756, -44.0

site 30 : energy = [-44.12735258514953, -44.0865735034287, -44.038963572097195] linkdim = 40 trunc = 1.4003788539204135e-5
site 29 : energy = [-44.1273232142111, -44.08655327696375, -44.03893115956366] linkdim = 40 trunc = 8.386701906287554e-6
site 28 : energy = [-44.127309348356285, -44.086528213603884, -44.038913441707464] linkdim = 40 trunc = 1.4464764057018943e-5
site 27 : energy = [-44.12727949053352, -44.08650558065976, -44.03887963346366] linkdim = 40 trunc = 8.326070387141934e-6
site 26 : energy = [-44.12726450550644, -44.08648175524651, -44.03886066732583] linkdim = 40 trunc = 1.4528152963771214e-5
site 25 : energy = [-44.1272384898407, -44.08645504990005, -44.0388269938876] linkdim = 40 trunc = 8.213436905023059e-6
site 24 : energy = [-44.127222414172714, -44.08643290170272, -44.03880707625024] linkdim = 40 trunc = 1.3865088735215489e-5
site 23 : energy = [-44.12719982460468, -44.086403679702684, -44.038775947520925] linkdim = 40 trunc = 8.027960594257623e-6
site 22 : energy 

┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31
┌ Warning: Calling `inner(x::MPS, A::MPO, y::MPS)` where the site indices of the `MPS`
│ `x` and the `MPS` resulting from contracting `MPO` `A` with `MPS` `y` don't
│ match is deprecated as of ITensors v0.3 and will result in an error in ITensors
│ v0.4. The most common cause of this is something like the following:
│ 
│ ```julia
│ s = siteinds("S=1/2")
│ psi = randomMPS(s)
│ H = MPO(s, "Id")
│ inner(psi, H, psi)
│ ```
│ 
│ `psi` ha

The 0th excited state energy is: -44.12691624495601
The 1th excited state energy is: -44.08614469533544
The 2th excited state energy is: -44.03848630356666
